# Ocean Plastic Modelling Tutorial

## How to use this tutorial:

At no point will you need to actually write code - you can interact with this tutorial by playing around with parameters in the already-written code. The one thing you have to be able to do is run the code in this notebook. The 'notebook' is the page you are on - it is made up of cells of text (like what you are reading now), and cells of code (grey regions with coloured fonts or 'Show code' written in blue if the code is hidden). To run the code in a cell, you simply click on the 'play' icon that shows up in the top left when you hover your cursor over the cell. Try running the cell below:

In [ ]:
print('Well done, you just ran your first cell of Python code!')

As you can see after running the cell, an output is printed below the code. The output of the code above is just the message that was specified to be printed, but we can do more useful things in the code and output the results.

The code below multiplies one number by another and outputs the result. Specify two numbers you want to multiply by replacing the '2' and '3' and then run the cell to print the result:

In [ ]:
number_1 = 2 # Insert any number here
number_2 = 3 # Insert any number here
product = number_1 * number_2
print(f"{number_1} times {number_2} is {product}")

## Overview

This tutorial focuses on modelling the motion of plastic in the oceans. Such models are created by non-profits like The Ocean Cleanup to help them make informed decisions about where to focus their cleanup efforts and which plastic extraction methods will be most effective.

Fluid dynamics will be an important topic in this tutorial, with concepts like advection, Eulerian and Lagrangian methods and numerical integration being central in these models.

We will recreate the 'advection scheme' behind the Python library 'Ocean Parcels', then use the library in some simple situations and see some more realistic use cases as laid out by Ocean Parcels' documentation.

## Advection
This notion is key in ocean plastic modelling because it describes exactly the behaviour we want to model. Advection is the transport of something by the motion of a fluid into which it is introduced. In the case of ocean plastic, the plastics whose motion we are trying to predict are advected by ocean currents and wind.

Essentially, advection states that when particles of a fluid are placed in a moving fluid, the particles of the newly introduced fluid will stay stationary with respect to the fluid around them - from the perspective of somebody watching from outside of the fluid, this means that the newly introduced fluid will move with the same flow as the fluid into which it has been introduced.

We will think of plastic in the ocean as a fluid - ocean plastic is made up of many small particles which can move around independently and mix, so it has all the properties of what we call fluids.

## Euler vs. Lagrange

There are two main ways of considering fields in compuational fluid dynamics - the Eulerian and Lagrangian viewpoints.

The Eulerian viewpoint considers a grid dividing up the space into cells. Each cell then has a set of values associated with it, representing the values of the fluid's properties. Taking the grid to be made up of infinitely small cells gives an exact description of the fluid, but on a computer we have to decide on cells that are 'small enough'.

The Lagrangian viewpoint of a fluid considers it as a collection of many parcels of the fluid. In this case, an exact description of the fluid would involve taking infinitely many parcels but we have to content ourselves with a 'big enough' number of parcels in practice.

In theory, either viewpoint effectively describes the fluid - but we have to decide on our approach before we start working on our problem. The approach that will be taken here is as follows:

- The velocity field (the flow of ocean currents) will be represented by an Eulerian field. This means we will have some part of the ocean split up into cells, with each cell having an arrow (called a vector) associated with it pointing in the direction of the flow of currents at that location in the ocean and with size proportional to the speed of the flow at that location.

- The fluid of plastic particles will be represented by a Lagrangian set of parcels.

This is the setup used in the Ocean Parcels library which we will use later. The choice is advantageous because it allows for existing datasets of ocean currents in the Eulerian specification to be used to produce results for the trajectories of discrete plastic particles which are most intuitively visualised in the Lagrangian specification.

## Coding the advection of fluid parcels in an Eulerian specified field

This method is the workhorse behind the models in the Ocean Parcels library. Before we start calling it from the library, let's figure out how we can create it ourselves.

Run the following cell - it imports the necessary libraries and sets up an Eulerian specification of a spiraling time-independent velocity field in a 2D square region broken into a 20$\times$20 grid:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the grid
grid_size = 20
x = np.linspace(0, 1, grid_size)
y = np.linspace(0, 1, grid_size)
X, Y = np.meshgrid(x, y)

# Define a spiraling velocity field (time-independent)
U = -(Y - 0.5)  # x-component of velocity
V = (X - 0.5)   # y-component of velocity

# Plot the velocity field
plt.figure(figsize=(4, 4))
plt.quiver(X, Y, U, V)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Velocity Field')
plt.show()

Now, we want to set up a plastic parcel whose trajectory we will simulate. Trajectory simulation is done using 'numerical integration'. The process of numerical integration is to decide on a small time step and using the speed and direction of currents at the starting location, figure out where the particle would be after the small time step (using speed $\times$ time = distance). Then, we use the speed and direction of the current at the new location to find the location of the particle after the next time step and repeat the process thousands of times to follow the particle's trajectory, with smaller time steps giving more accurate results than larger time steps.

Two variables are defined below: *time_step* (to store the size of the time step) and *num_steps* (to store the number of steps that will be taken). Then we define a numpy array of zeros, *parcel_trajectory*, that has shape (*num_steps* +1)$\times 2$. To start our parcel at (0.6,0.4), the first row in the array is given that value.

Then we can carry out the numerical integration of the trajectory. The second cell below (with the code hidden) does this with a particularly precise method of making the small time steps called 'fourth-order Runge-Kutta'.

You can run both cells to see the trajectory of a particle in the fluid and change around parameters in the first cell then run them both again to see how changing the start point, time step and number of steps affects the outputted trajectory:


In [ ]:
# Parameters
time_step = 0.1
num_steps = 30

# Initialize the parcel trajectory array
parcel_trajectory = np.zeros((num_steps + 1, 2))

# Starting position of the parcel
parcel_trajectory[0] = np.array([0.6, 0.4])


In [ ]:
# @title
# Function to get velocity in the grid cell where the parcel currently is:
def get_velocity(position, U, V, x, y):
    # Find the nearest indices in the grid
    ix = np.argmin(np.abs(x - position[0]))
    iy = np.argmin(np.abs(y - position[1]))
    return np.array([U[iy, ix], V[iy, ix]])

# Runge-Kutta integration
for n in range(num_steps):
    r_n = parcel_trajectory[n]

    k1 = get_velocity(r_n, U, V, x, y)
    k2 = get_velocity(r_n + time_step * k1 / 2, U, V, x, y)
    k3 = get_velocity(r_n + time_step * k2 / 2, U, V, x, y)
    k4 = get_velocity(r_n + time_step * k3, U, V, x, y)

    r_next = r_n + (time_step / 6) * (k1 + 2 * k2 + 2 * k3 + k4)

    # Update the trajectory
    parcel_trajectory[n + 1] = r_next

# Plot the velocity field
plt.figure(figsize=(4, 4))
plt.quiver(X, Y, U, V, color='gray', alpha=0.6)

# Plot the parcel trajectory
plt.plot(parcel_trajectory[:, 0], parcel_trajectory[:, 1], label='Parcel Trajectory', marker='None')
plt.scatter([0.6], [0.4], color='blue', s=50, label='Start Point')

plt.xlabel('x')
plt.ylabel('y')
plt.title('Parcel Trajectory on Velocity Field')
plt.legend()
plt.show()


# Many Parcel Simulations - Ocean Parcels library
Now that we have figured out how to interpret fluid advection mathematically and how to numerically integrate to get the trajectory of a single parcel in a fluid of plastic particles in an ocean current, it's time to try to numerically integrate the motion of a group of parcels to discover the motion of the fluid of plastic particles as a whole.

As mentioned earlier, Ocean Parcels is the library that this tutorial will walk you through using to achieve this goal. It uses fourth-order Runge-Kutta to numerically integrate the advection equation of parcels, as we just did ourselves and has extra structure so that we can easily apply the same process to large groups of parcels.

To start with, run the following cell to install Ocean Parcels and import all of the neccessary packages. This could take about 30 seconds.

In [ ]:
# @title
%%capture
!pip install parcels
!pip install trajan
!pip install xarray

import math
from datetime import timedelta
from operator import attrgetter

import matplotlib.pyplot as plt
import numpy as np
import trajan as ta
import xarray as xr
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
from matplotlib import rc

import parcels

Now, run the following cell to retrieve a folder of example data from Ocean Parcels. We will use currents from this dataset as the currents which our plastic particles are exposed to. Run the following cell to view the flows of these currents at an initial time and a later time.

In [ ]:
# @title
example_dataset_folder = parcels.download_example_dataset("MovingEddies_data")

fieldset = parcels.FieldSet.from_parcels(f"{example_dataset_folder}/moving_eddies")

plt.figure(figsize=(10, 6))
for t in [0.0, 600000.0]:
  fieldset.computeTimeChunk(time=t)

  # Get the grid and data
  lon = fieldset.U.grid.lon  # 1D array of longitudes
  lat = fieldset.U.grid.lat  # 1D array of latitudes
  U = fieldset.U.data[0, :, :]  # Zonal component (2D array)
  V = fieldset.V.data[0, :, :]  # Meridional component (2D array)

  # Create a 2D meshgrid
  lon_grid, lat_grid = np.meshgrid(lon, lat)

  # Determine the indices to sample
  x_indices = np.linspace(0, lon_grid.shape[1] - 1, 20, dtype=int)
  y_indices = np.linspace(0, lon_grid.shape[0] - 1, 35, dtype=int)

  # Use these indices to select a subset of the data
  quiver_lon = lon_grid[y_indices[:, None], x_indices]
  quiver_lat = lat_grid[y_indices[:, None], x_indices]
  quiver_U = U[y_indices[:, None], x_indices]
  quiver_V = V[y_indices[:, None], x_indices]

  # Create the quiver plot
  if t == 0.0:
    plt.quiver(quiver_lon, quiver_lat, quiver_U, quiver_V, scale=30, color = 'blue', label = 'Initial Time')
  else:
    plt.quiver(quiver_lon, quiver_lat, quiver_U, quiver_V, scale=30, color = 'red', label = 'Later Time')

  # Add labels and title
  plt.xlabel("Zonal distance [m]")
  plt.ylabel("Meridional distance [m]")
  plt.title("Quiver Plot of the Vector Field")
  plt.legend()

  # Optionally set aspect ratio
  plt.gca().set_aspect('equal')

plt.show()


Now, let's put some parcels in this velocity field and track their trajectories. Run the following cell to set up two particles in the currents and view their trajectories. Again, you can edit the parameters and add more particles and see how it affects the trajectories (note that the start positions are in 100s of kilometers, so hundreds of thousands of meters).

In [ ]:
pset = parcels.ParticleSet.from_list(
    fieldset=fieldset,  # the fields on which the particles are advected
    pclass=parcels.JITParticle,  # the type of particles (JITParticle or ScipyParticle)
    lon=[3.3e5, 3.3e5],  # a set of release longitudes
    lat=[1e5, 2.8e5],  # a set of release latitudes
)

output_file = pset.ParticleFile(
    name="EddyParticles.zarr",
    outputdt=timedelta(hours=1), # how often we save a point along the trajectory in order to show it later
)
pset.execute(
    parcels.AdvectionRK4,  # this is the 4th order Runge-Kutta Kernel
    runtime=timedelta(days=6), # how long we let the simulation run for
    dt=timedelta(minutes=5), # the small time step in the numerical integration
    output_file=output_file,
)

ds = xr.open_zarr("EddyParticles.zarr")
plt.plot(ds.lon.T, ds.lat.T, ".-")
plt.xlabel("Zonal distance [m]")
plt.ylabel("Meridional distance [m]")
plt.show()

The following cell outputs an animation of the parcels' movement - run it to see the animation for your particles.

In [ ]:
# @title
fig = plt.figure(figsize=(3.5, 2.5), constrained_layout=True)
ax = fig.add_subplot()

ax.set_ylabel("Meridional distance [m]")
ax.set_xlabel("Zonal distance [m]")
ax.set_xlim(0, 4e5)
ax.set_ylim(0, 7e5)

# show only every fifth output (for speed in creating the animation)
timerange = np.unique(ds["time"].values)[::5]

# Indices of the data where time = 0
time_id = np.where(ds["time"] == timerange[0])

sc = ax.scatter(ds["lon"].values[time_id], ds["lat"].values[time_id])

t = str(timerange[0].astype("timedelta64[h]"))
title = ax.set_title(f"Particles at t = {t}")

plt.close(fig)

def animate(i):
    t = str(timerange[i].astype("timedelta64[h]"))
    title.set_text(f"Particles at t = {t}")

    time_id = np.where(ds["time"] == timerange[i])
    sc.set_offsets(np.c_[ds["lon"].values[time_id], ds["lat"].values[time_id]])


anim = FuncAnimation(fig, animate, frames=len(timerange), interval=100)
HTML(anim.to_jshtml())

# Using Real Ocean Data

Now that we have seen how to use Ocean Parcels to obtain the trajectories of groups of parcels, let's apply it using real world ocean current data.

The following cell loads data about currents in the ocean around South Africa from [GlobCurrents](https://www.ifremer.fr/fr):

In [ ]:
# @title
example_dataset_folder = parcels.download_example_dataset("GlobCurrent_example_data")

filenames = {
    "U": f"{example_dataset_folder}/20*.nc",
    "V": f"{example_dataset_folder}/20*.nc",
}

variables = {
    "U": "eastward_eulerian_current_velocity",
    "V": "northward_eulerian_current_velocity",
}

dimensions = {"lat": "lat", "lon": "lon", "time": "time"}

fieldset = parcels.FieldSet.from_netcdf(filenames, variables, dimensions)

Now, the following cell releases twenty particles in a line of constant latitude at -33 and going from 28 to 30 in longitude, then simulates their motion and shows their trajectories on a map:

In [ ]:
pset_new = parcels.ParticleSet.from_line(
    fieldset=fieldset,
    pclass=parcels.JITParticle,
    size=20,  # releasing 20 particles
    start=(28, -33),  # releasing on a line: the start longitude and latitude
    finish=(30, -33),  # releasing on a line: the end longitude and latitude
)

output_file_new = pset.ParticleFile(
    name="output.zarr", outputdt=timedelta(hours=6)
)

pset_new.execute(
    parcels.AdvectionRK4,
    runtime=timedelta(days=10),
    dt=timedelta(minutes=5),
    output_file=output_file_new,
)

ds = xr.open_zarr("output.zarr")
ds.traj.plot(margin=2)
plt.show()

We have now simulated the motion of ocean plastics! The next step to more accurately simulate the trajectories of plastic particles would be to include eddy diffusitivty - the movement of the particle due to currents on scales smaller than the cell size of the Eulerian grid. Including this in the simulation isn't difficult, we would just randomly displace the particle by a small amount at the end of each small step. Beyond this, the inclusion of wind effects and three dimensional models where the depth of particles is also taken into account would add to the precision of the predictions that can be made with the simulations.

If you have enjoyed working through this tutorial, I would recommend having a look at [this article on The Ocean Cleanup's website](https://theoceancleanup.com/updates/forecasting-ocean-plastic-around-the-globe-a-deep-dive-into-modeling-the-garbage-patches/) on the topic and perhaps getting to grips with Python to explore using OceanParcels independently!